''' This script uses the pyomo optimization library and ipopt solver
    to fit water advancement data based on the Y = X ^ r equation.
    This equation is used to model the advancement of water in border irrigation.
    X represents advancement time (min) and Y represents advancement length (m)
'''

## Importing the necessary libraries

In [ ]:
from pyomo.environ import *
import matplotlib.pyplot as plt

## Creating the abstract pyomo model

In [ ]:
# The Objective Function based on the Least Squared Error (LSE)
def rule_OF(model):
    return model.OF==sum((model.y[i] - (model.x[i])**model.r)**2 for i in model.N)/len(model.N)

# Creating the Abstract Model
model = AbstractModel()
model.N = Set()
model.M = Set()
model.x=Param(model.N)
model.y=Param(model.M)

# Defining the boundaries of decision variable r
model.r = Var(bounds=(0,1),within=Reals) 

# Defining the Optimization Function as a variable
model.OF = Var(within=NonNegativeReals, initialize=1)
model.C   = Constraint(rule=rule_OF)

# Setting the optimization model to minimize the LSE formula
model.obj1 = Objective(expr=model.OF, sense=minimize)

## Choosing the solver, inputing data and solving the model

In [ ]:
opt = SolverFactory('Ipopt') # ipopt solver is used for a non linear equation
opt.options['max_iter'] = 3000
instance = model.create_instance('Data.dat')
results = opt.solve(instance, tee=True) # solves and updates instance

## Printing the results

In [ ]:
print('OF= ',value(instance.obj1))
print('r= ',value(instance.r))

## Plotting the curve against the data

In [ ]:
fig = plt.figure(figsize=(7,7))

xc = []
yc = []
for i in instance.N:
    yc.append((value(instance.x[i])**(value(instance.r))))
    xc.append(value(instance.x[i]))
    plt.scatter(value(instance.x[i]), value(instance.y[i]))
              
plt.plot(xc,yc,'-r', label='Y=X^r')
plt.title('Graph of Y=X^r')
plt.legend(loc='upper left')
plt.xlabel('X')
plt.ylabel('Y');